In [2]:
import random
import time
import numpy as np
from collections import namedtuple, Counter, defaultdict
import math
import functools 
import time

cache = functools.lru_cache(10**6)

random.seed(109)

# MiniMax Mancala Project 

Below is a simple implementation of Mancala from Homework 7 with modifications. I made a class Game for Mancala to inherit. It's methods will allow for MinMax to be implemented easier

In [3]:
class Game:
    def is_terminal(self, state):
        raise NotImplementedError
    
    def actions(self, state):
        raise NotImplementedError

    def result(self, state, move):
        raise NotImplementedError
    
    def utility(self, state, player):
        raise NotImplementedError

In [4]:
class Mancala(Game):
    def __init__(self, pits_per_player=3, stones_per_pit = 2, toggle_random_generator_p2 = False, cap_on_moves = None, player1_name = "Player 1", player2_name = "Player 2"):
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.player1_name = player1_name
        self.player2_name = player2_name
        self.current_player = 1
        self.moves = []
        
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1

        self.toggle_random_generator_p2 = toggle_random_generator_p2
        self.cap_on_moves = cap_on_moves
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0



        
    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)



        
    def valid_move(self, pit):

        # Pit falls in range
        if not (1 <= pit <= self.pits_per_player):
            return False
        
        # Check if P1 pit is empty or P2
        if self.current_player == 1:
            return self.board[pit - 1] > 0
        elif self.current_player == 2:
            return self.board[self.p2_pits_index[0] + pit - 1] > 0

        return False
    


        
    def random_move_generator_p2(self):
        valid_moves = []
        
        # Range
        start_index = self.p2_pits_index[0]
        end_index = self.p2_pits_index[1]
        # Look for valid moves
        for i in range(start_index, end_index + 1):
            if self.board[i] > 0:
                valid_moves.append(i - start_index + 1)

        
        # Return random valid move
        return random.choice(valid_moves) if valid_moves else None
        pass




    def play(self, pit = None):
        if not self.is_terminal():
            if self.current_player == 1:
                if pit is None or self.valid_move(pit) == False:
                    return f"Invalid Move by Player {self.current_player}"
   
                self.executePlay(pit)
                self.moves.append((1, pit))
                # print(f"{self.player1_name} chose pit: {pit}")
                
                
            elif self.current_player == 2:
                if self.toggle_random_generator_p2:
                    p2_choice = self.random_move_generator_p2()
                    self.executePlay(p2_choice)
                    self.moves.append((2, p2_choice))
                    # print(f"{self.player2_name} chose pit: {p2_choice}")
                else:
                    if self.valid_move(pit) == False:
                        return f"Invalid Move by Player {self.current_player}"
                        
                    self.executePlay(pit)
                    self.moves.append((2, pit))
                    # print(f"{self.player2_name} chose pit: {pit}")
        else:
            print("GAME END")
            self.print_game_results()
            
                
        # Switch players
        self.current_player = 3 - self.current_player
    



    def executePlay(self, pit):
        # Grab pit index
        if self.current_player == 1:
            index = pit - 1
        if self.current_player == 2:
            index = self.p2_pits_index[0] + pit - 1
        
        
        # Save and clear current index stones
        stones = self.board[index]
        self.board[index] = 0
        
        next_index = index
        while stones > 0:
            # Iterate index
            next_index = (next_index + 1) % len(self.board)
            # Skip opposite Mancala
            if (self.current_player == 1 and next_index == self.p2_mancala_index) or (self.current_player == 2 and next_index == self.p1_mancala_index):
                continue
            # Reduce stone and increase 1 on whatever pit
            self.board[next_index] += 1
            stones -= 1    


    """ Switched winning_eval -> is_terminal. Readjusted it """
    def is_terminal(self, state=None):
        """ Check if the game is over """
        if state is None:
            state = self

        # Check for a completely empty player side
        p1_empty = all(state.board[i] == 0 for i in range(state.p1_pits_index[0], state.p1_pits_index[1] + 1))
        p2_empty = all(state.board[i] == 0 for i in range(state.p2_pits_index[0], state.p2_pits_index[1] + 1))
        
        # In the instance of a cap we look at
        if state.cap_on_moves is not None:
            is_cap_moves_reached = len(state.moves) >= state.cap_on_moves
            return p1_empty or p2_empty or is_cap_moves_reached
        
        return p1_empty or p2_empty

    def print_final_game_results(self):
        assert self.is_terminal() == True

        remaining_stones_p1 = sum(self.board[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        remaining_stones_p2 = sum(self.board[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        sum_p1 = remaining_stones_p1 + self.board[self.p1_mancala_index]
        sum_p2 = remaining_stones_p2 + self.board[self.p2_mancala_index]

        print("Leftover Stones...")
        print("------------------------------")
        print(f"{self.player1_name} Remaining Stones: {remaining_stones_p1}")
        print(f"{self.player2_name} Remaining Stones: {remaining_stones_p2}")
        print("------------------------------")

        print("Final Results!")
        print("------------------------------")
        print(f"{self.player1_name} Score: {sum_p1}")
        print(f"{self.player2_name} Score: {sum_p2}")
        print("------------------------------")

        if sum_p1 > sum_p2:
            print(f"{self.player1_name} Wins!")
            return 1
        elif sum_p1 < sum_p2:
            print(f"{self.player2_name} Wins!")
            return 2
        else:
            print("It's a Tie!")
            return 0
    
    def get_winner(self):
        remaining_stones_p1 = sum(self.board[i] for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        remaining_stones_p2 = sum(self.board[i] for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        sum_p1 = remaining_stones_p1 + self.board[self.p1_mancala_index]
        sum_p2 = remaining_stones_p2 + self.board[self.p2_mancala_index]

        if sum_p1 > sum_p2:
            return 1
        elif sum_p1 < sum_p2:
            return 2
        
        # Neither
        return 0




    """
    Game Logic Methods for MinMax/AlphaBeta
    """
    
    def actions(self, state=None):
        """ Return list of valid moves for the current player """
        if state is None:
            state = self

        # Determine which side of the board to check based on current player and check for pits with stones
        if state.current_player == 1:
            start_index = state.p1_pits_index[0]
            end_index = state.p1_pits_index[1]
            valid_moves = [i - start_index + 1 for i in range(start_index, end_index + 1) if state.board[i] > 0]
        else:
            start_index = state.p2_pits_index[0]
            end_index = state.p2_pits_index[1]
            valid_moves = [i - start_index + 1 for i in range(start_index, end_index + 1) if state.board[i] > 0]
        
        return valid_moves

    def result(self, state, move):
        """ Create a new state after making a move """
        # Create a copy of the current state
        new_state = Mancala(
            pits_per_player=self.pits_per_player, 
            stones_per_pit=0,
            toggle_random_generator_p2=self.toggle_random_generator_p2,
            cap_on_moves=self.cap_on_moves
        )
        new_state.board = state.board.copy()
        new_state.current_player = state.current_player
        new_state.moves = state.moves.copy()



        # Simulate the move
        if new_state.current_player == 1:
            index = move - 1
        else:
            index = new_state.p2_pits_index[0] + move - 1

            
        
        # Grab pit index stones
        stones = new_state.board[index]
        new_state.board[index] = 0
        
        next_index = index
        while stones > 0:
            # Iterate index
            next_index = (next_index + 1) % len(new_state.board)
            
            # Skip opposite Mancala
            if (new_state.current_player == 1 and next_index == new_state.p2_mancala_index) or \
               (new_state.current_player == 2 and next_index == new_state.p1_mancala_index):
                continue
            
            # Reduce stone and increase 1 on whatever pit
            new_state.board[next_index] += 1
            stones -= 1
        
        # Switch players
        new_state.current_player = 3 - new_state.current_player
        
        return new_state


    def utility(self, state, player):
        """ Calculate the utility of the state for a player by looking at how much stones their side has"""
        if state is None:
            state = self

        # Calculate total stones for each player
        player1_stones = state.board[state.p1_mancala_index] + \
            sum(state.board[i] for i in range(state.p1_pits_index[0], state.p1_pits_index[1] + 1))
        
        player2_stones = state.board[state.p2_mancala_index] + \
            sum(state.board[i] for i in range(state.p2_pits_index[0], state.p2_pits_index[1] + 1))

        # Return utility from the perspective of the specified player
        return player1_stones - player2_stones if player == 1 else player2_stones - player1_stones

# Alpha/Beta Pruning and MiniMax
The Minimax algorithm w/ alpha/beta pruning below is what is going to be used

In [5]:
def minimax_search(game, state):
    """ MinMax Search """

    player = state.current_player
    infinity = np.inf

    def max_value(state, alpha, beta):
        # If game is over return the utilty, breaks recursion
        if game.is_terminal(state):
            return game.utility(state, player), None
        
        v, move = -infinity, None

        # Looking at all actions at said state
        for a in game.actions(state):
            # For each actions find it's min val recursively
            v2, _ = min_value(game.result(state, a), alpha, beta)
            # Set new alpha
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)

        return v, move

    def min_value(state, alpha, beta):
        # If game is over return the utilty, breaks recursion
        if game.is_terminal(state):
            return game.utility(state, player), None
        
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)

        return v, move
    
    # Start off w/ finding max outcome for P1
    return max_value(state, -infinity, +infinity)

In [6]:
def alphabeta_search(game, state):
    """ Alpha-Beta Pruning Search """

    player = state.current_player
    infinity = np.inf

    def max_value(state, alpha, beta):
        # If game is over return the utilty, breaks recursion
        if game.is_terminal(state):
            return game.utility(state, player), None
        
        v, move = -infinity, None

        # Looking at all actions at said state
        for a in game.actions(state):
            # For each actions find it's min val recursively
            v2, _ = min_value(game.result(state, a), alpha, beta)
            # Set new alpha
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            # Pruning
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        # If game is over return the utilty, breaks recursion
        if game.is_terminal(state):
            return game.utility(state, player), None
        
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            # Pruning
            if v <= alpha:
                return v, move
        return v, move
    
    # Start off w/ finding max outcome for P1
    return max_value(state, -infinity, +infinity)

# Intermediate Stage

For this stage while I currently have the general layout of the game algorithm to be used and adjusted Mancala to inherit from the Game super class but work needs to be done. Here is a list that I aim to work with:

* Run test game w/ person vs AI  -> this will let me actually understand MinMax performance
* Adjust Mancala utility function -> after testing whats above and some research I'll adjust heuristics as needed


In [7]:
# Current makes a random choice on valid moves 
def random_player(game, state):
    """ Random move selection player """
    return random.choice(game.actions(state))

def minimax_player(game, state):
    """ MiniMax player using alpha-beta search """
    return minimax_search(game, state)[1]

def alphabeta_player(game, state):
    """ MiniMax player using alpha-beta search """
    return alphabeta_search(game, state)[1]


# Test

Below are just simple tests to look at the instances of two rando duking it out verse minmax against rando. A key insight I see here is that despite Rando1 vs Rando2 is like what it is, random. In most instances Rando1 still wins. It may be for favorable to go first in the Mancala game.

In [8]:
# Single game

mancala = Mancala(player1_name="Rando1", player2_name="Rando2")
player_1_wins = 0
player_2_wins = 0


# Test Run w/ Rando
while not mancala.is_terminal():
    mancala.display_board()
    # Both players random
    move = random_player(mancala, mancala)

    print(f"Player {mancala.current_player} moves: {move}")
    mancala.play(move)

# Print final results
mancala.print_final_game_results()
        

P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 2 | 2 | <- 2
3 -> |_2_|_2_| <- 1
         0         
Turn: P1
Player 1 moves: 2
P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 0 | 2 | <- 2
3 -> |_3_|_2_| <- 1
         1         
Turn: P2
Player 2 moves: 1
P1               P2
     ____0____     
1 -> | 2 | 3 | <- 3
2 -> | 0 | 3 | <- 2
3 -> |_3_|_0_| <- 1
         1         
Turn: P1
Player 1 moves: 3
P1               P2
     ____0____     
1 -> | 2 | 3 | <- 3
2 -> | 0 | 4 | <- 2
3 -> |_0_|_1_| <- 1
         2         
Turn: P2
Player 2 moves: 2
P1               P2
     ____1____     
1 -> | 3 | 4 | <- 3
2 -> | 1 | 0 | <- 2
3 -> |_0_|_1_| <- 1
         2         
Turn: P1
Player 1 moves: 1
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 2 | 0 | <- 2
3 -> |_1_|_1_| <- 1
         3         
Turn: P2
Player 2 moves: 1
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 2 | 1 | <- 2
3 -> |_1_|_0_| <- 1
         3        

1

In [9]:
mancala = Mancala(player1_name="Rando1", player2_name="Rando2")
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 iterations of random
for _ in range(0,100):
    while not mancala.is_terminal():        
        # Both players random
        move = random_player(mancala, mancala)
        mancala.play(move)

    # Print final results
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="Rando1", player2_name="Rando2")
    

In [10]:
print("Stats:")
print ("----------------------------------")
print(f"Rando 1 total wins {player_1_wins}")
print(f"Rando 2 total wins {player_2_wins}")
print(f"Ties: {ties}")
print ("----------------------------------")

Stats:
----------------------------------
Rando 1 total wins 44
Rando 2 total wins 40
Ties: 16
----------------------------------


In [11]:
# Single MinMax
mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando")

# Test Run w/ MinMax
while not mancala.is_terminal():
    mancala.display_board()

    if mancala.current_player == 1:
        # Player 1 chooses move
        move = minimax_player(mancala, mancala)
    else:
        # Player 2 can be random or MiniMax
        move = random_player(mancala, mancala)

    print(f"Player {mancala.current_player} moves: {move}")
    mancala.play(move)

# Print final results
mancala.print_final_game_results()


P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 2 | 2 | <- 2
3 -> |_2_|_2_| <- 1
         0         
Turn: P1


Player 1 moves: 2
P1               P2
     ____0____     
1 -> | 2 | 2 | <- 3
2 -> | 0 | 2 | <- 2
3 -> |_3_|_2_| <- 1
         1         
Turn: P2
Player 2 moves: 1
P1               P2
     ____0____     
1 -> | 2 | 3 | <- 3
2 -> | 0 | 3 | <- 2
3 -> |_3_|_0_| <- 1
         1         
Turn: P1
Player 1 moves: 1
P1               P2
     ____0____     
1 -> | 0 | 3 | <- 3
2 -> | 1 | 3 | <- 2
3 -> |_4_|_0_| <- 1
         1         
Turn: P2
Player 2 moves: 2
P1               P2
     ____1____     
1 -> | 1 | 4 | <- 3
2 -> | 1 | 0 | <- 2
3 -> |_4_|_0_| <- 1
         1         
Turn: P1
Player 1 moves: 1
P1               P2
     ____1____     
1 -> | 0 | 4 | <- 3
2 -> | 2 | 0 | <- 2
3 -> |_4_|_0_| <- 1
         1         
Turn: P2
Player 2 moves: 3
Leftover Stones...
------------------------------
MinMaxMan Remaining Stones: 9
Rando Remaining Stones: 0
------------------------------
Final Results!
------------------------------
MinMaxMan Score: 10
Rando Score: 2
-----------------------------

1

# MINMAX TEST W/ 3,5,10 PLIES

In [12]:
start_time = time.time()

mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando", cap_on_moves=3)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 MinMax
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = minimax_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando")

end_time = time.time()


In [13]:
print("Stats:")
print ("----------------------------------")
print(f"MinMaxMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time : {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
MinMaxMan total wins 92
Rando total wins 0
Ties: 8
Time : 1.0946716284751892 seconds
----------------------------------


In [14]:
start_time = time.time()

mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando", cap_on_moves=5)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 MinMax
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = minimax_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando")

end_time = time.time()


In [15]:
print("Stats:")
print ("----------------------------------")
print(f"MinMaxMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time : {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
MinMaxMan total wins 86
Rando total wins 1
Ties: 13
Time : 1.1281112098693848 seconds
----------------------------------


In [16]:
start_time = time.time()

mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando", cap_on_moves=10)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 MinMax
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = minimax_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando")

end_time = time.time()


In [17]:
print("Stats:")
print ("----------------------------------")
print(f"MinMaxMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time : {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
MinMaxMan total wins 91
Rando total wins 0
Ties: 9
Time : 1.1140214967727662 seconds
----------------------------------


In [18]:
start_time = time.time()

mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando", cap_on_moves=10)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 MinMax
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = minimax_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="MinMaxMan", player2_name="Rando")

end_time = time.time()


In [19]:
print("Stats:")
print ("----------------------------------")
print(f"MinMaxMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time : {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
MinMaxMan total wins 87
Rando total wins 0
Ties: 13
Time : 1.1240146660804748 seconds
----------------------------------


# Notes
* MinMax does signficiantly better, that is always winning, than the Random player. Though it does take much longer to run..

# ALPHA BETA W/ 3,5,10 PLIES

In [20]:
start_time = time.time()

mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando", cap_on_moves=3)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 AlphaBeta
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = alphabeta_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando")
    end_time = time.time()


In [21]:
print("Stats:")
print ("----------------------------------")
print(f"AlphaBetaMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time per game: {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
AlphaBetaMan total wins 89
Rando total wins 0
Ties: 11
Time per game: 0.016119868755340577 seconds
----------------------------------


In [22]:
start_time = time.time()

mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando", cap_on_moves=5)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 AlphaBeta
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = alphabeta_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando")
    end_time = time.time()


In [23]:
print("Stats:")
print ("----------------------------------")
print(f"AlphaBetaMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time per game: {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
AlphaBetaMan total wins 85
Rando total wins 1
Ties: 14
Time per game: 0.01586367130279541 seconds
----------------------------------


In [24]:
start_time = time.time()

mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando", cap_on_moves=10)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 AlphaBeta
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = alphabeta_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando")
    end_time = time.time()


In [25]:
print("Stats:")
print ("----------------------------------")
print(f"AlphaBetaMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time per game: {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
AlphaBetaMan total wins 89
Rando total wins 0
Ties: 11
Time per game: 0.016527905464172363 seconds
----------------------------------


In [26]:
start_time = time.time()

mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando", cap_on_moves=10)
player_1_wins = 0
player_2_wins = 0
ties = 0


# Test Run w/ 100 AlphaBeta
for _ in range(100):
    while not mancala.is_terminal():

        if mancala.current_player == 1:
            # Player 1 chooses move
            move = alphabeta_player(mancala, mancala)
        else:
            # Player 2 can be random or MiniMax
            move = random_player(mancala, mancala)

        mancala.play(move)
    winner = mancala.get_winner()
    
    if winner == 1:
        player_1_wins += 1
    elif winner == 2:
        player_2_wins += 1
    else:
        ties += 1
    
    mancala = Mancala(player1_name="AlphaBetaMan", player2_name="Rando")
    end_time = time.time()


In [27]:
print("Stats:")
print ("----------------------------------")
print(f"AlphaBetaMan total wins {player_1_wins}")
print(f"Rando total wins {player_2_wins}")
print(f"Ties: {ties}")
print(f"Time per game: {(end_time - start_time) / 100} seconds")
print ("----------------------------------")

Stats:
----------------------------------
AlphaBetaMan total wins 85
Rando total wins 0
Ties: 15
Time per game: 0.01571658134460449 seconds
----------------------------------


# Summary

* Alpha-beta perforces signficantly faster though in terms of win rate both algorithms seem to achieve similar win rates. This makes sense as alpha-beta should just yield the same result but without needing to explore any other terminal nodes.